## detectron2

In [20]:
import sys
sys.path.append("./detectron2")
import detectron2 
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

## densepose

In [65]:
import sys
sys.path.append("./detectron2")
sys.path.append("./detectron2/projects/DensePose")
from typing import List
from detectron2.data.detection_utils import read_image
from densepose import add_densepose_config
from densepose.structures import DensePoseChartPredictorOutput, DensePoseEmbeddingPredictorOutput
from densepose.utils.logger import verbosity_to_level
from densepose.vis.base import CompoundVisualizer
from densepose.vis.bounding_box import ScoredBoundingBoxVisualizer
from densepose.vis.densepose_outputs_vertex import (
    DensePoseOutputsTextureVisualizer,
    DensePoseOutputsVertexVisualizer,
    get_texture_atlases,
)
from densepose.vis.densepose_results import (
    DensePoseResultsContourVisualizer,
    DensePoseResultsFineSegmentationVisualizer,
    DensePoseResultsUVisualizer,
    DensePoseResultsVVisualizer,
)
from densepose.vis.densepose_results_textures import (
    DensePoseResultsVisualizerWithTexture,
    get_texture_atlas,
)
from densepose.vis.extractor import (
    CompoundExtractor,
    DensePoseOutputsExtractor,
    DensePoseResultExtractor,
    create_extractor,
)

class DensePoseModel:
    VISUALIZERS = {
        "dp_contour": DensePoseResultsContourVisualizer,
        "dp_segm": DensePoseResultsFineSegmentationVisualizer,
        "dp_u": DensePoseResultsUVisualizer,
        "dp_v": DensePoseResultsVVisualizer,
        "dp_vertex": DensePoseOutputsVertexVisualizer,
        "bbox": ScoredBoundingBoxVisualizer,
    }
        
    def __init__(self, config_path: str, model_path: str):
        print(f"load densepose model from {model_path}...")
        cfg = get_cfg()
        add_densepose_config(cfg)
        cfg.merge_from_file(config_path)
        cfg.MODEL.WEIGHTS = model_path
        cfg.freeze()
        self.cfg = cfg
        self.predictor = DefaultPredictor(cfg)
        print("finished loading")

    def _visualize_outputs(self, input_img, outputs, vis_specs:List[str], save_path: str, white_background: bool = False):
        # create visualizer and extractor
        visualizers = []
        extractors = []
        for vis_spec in vis_specs:
            vis = DensePoseModel.VISUALIZERS[vis_spec](
                cfg=self.cfg,
            )
            visualizers.append(vis)
            extractor = create_extractor(vis)
            extractors.append(extractor)
        visualizer = CompoundVisualizer(visualizers)
        extractor = CompoundExtractor(extractors)
    
        # visualize and save
        image = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        image = np.tile(image[:, :, np.newaxis], [1, 1, 3])
        if white_background:
            image.fill(255)
        data = extractor(outputs)
        image_vis = visualizer.visualize(image, data)
        cv2.imwrite(save_path, image_vis)

    def predict_and_save(self, input_img_path: str, output_img_path: str = None, vis_specs_string: str = "dp_contour,bbox"):
        if not output_img_path:
            suffix = input_img_path.split(".")[-1]
            output_img_path = input_img_path.replace(f".{suffix}", f"-{vis_specs}.{suffix}")
        vis_specs = vis_specs_string.split(",")
        img = read_image(input_img_path, format="BGR")  # predictor expects BGR image.
        with torch.no_grad():
            outputs = self.predictor(img)["instances"]
            self._visualize_outputs(img, outputs, vis_specs, output_img_path)
            print(f"output is saved to {output_img_path}")


In [66]:
model_path = "/home/bo/workspace/diffusions/models/densepose_rcnn_R_50_FPN_s1x.pkl"
config_path = "/home/bo/workspace/diffusions/detectron2/projects/DensePose/configs/densepose_rcnn_R_50_FPN_s1x.yaml"
input_img_path = "/home/bo/workspace/diffusions/images/frame_156.jpg"
vis_specs = "dp_contour,bbox"
output_img_path = input_img_path.replace(".jpg", f"-{vis_specs}.jpg")
model = DensePoseModel(config_path, model_path)

load densepose model from /home/bo/workspace/diffusions/models/densepose_rcnn_R_50_FPN_s1x.pkl...
[08/24 17:42:23 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /home/bo/workspace/diffusions/models/densepose_rcnn_R_50_FPN_s1x.pkl ...
finished loading


In [67]:
model.predict_and_save(input_img_path, output_img_path, vis_specs)

output is saved to /home/bo/workspace/diffusions/images/frame_156-dp_contour,bbox.jpg


/home/bo/workspace/diffusions/./detectron2/projects/DensePose/densepose/vis/densepose_results.py:135: UserWarning: The following kwargs were not used by contour: 'cfg'
  plt.contour(u, self.levels, extent=extent, **self.plot_args)
/home/bo/workspace/diffusions/./detectron2/projects/DensePose/densepose/vis/densepose_results.py:136: UserWarning: The following kwargs were not used by contour: 'cfg'
  plt.contour(v, self.levels, extent=extent, **self.plot_args)
